In [1]:
from pandas import ExcelFile
# import data
BMI_female_filereader = ExcelFile("BMI_female.xlsx")
BMI_female_dataframe = BMI_female_filereader.parse("Data")
BMI_male_filereader = ExcelFile("BMI_male.xlsx")
BMI_male_dataframe = BMI_male_filereader.parse("Data")
BP_female_filereader = ExcelFile("BP_female.xlsx")
BP_female_dataframe = BP_female_filereader.parse("Data")
BP_male_filereader = ExcelFile("BP_male.xlsx")
BP_male_dataframe = BP_male_filereader.parse("Data")
urban_pct_filereader = ExcelFile("urban_pct.xlsx")
urban_pct_dataframe = urban_pct_filereader.parse("Data")
continent_filereader = ExcelFile("continent.xls")
continent_dataframe = continent_filereader.parse("Country Summary Information")

In [2]:
# clean data

# year filter
urban_pct_dataframe = urban_pct_dataframe.filter(items = ["Urban population (% of total)"] +list(range(1980, 2009)))

# country filter
country_set1 = set(BMI_female_dataframe[BMI_female_dataframe.columns[0]])
country_set2 = set(urban_pct_dataframe[urban_pct_dataframe.columns[0]])
country_set3 = set(continent_dataframe[continent_dataframe.columns[0]])
countries = country_set1.intersection(country_set2, country_set3)

continent_dataframe = continent_dataframe[continent_dataframe[continent_dataframe.columns[0]].isin(countries)]
continent_dataframe = continent_dataframe.sort_values(continent_dataframe.columns[0])



In [3]:
# split datasets by continent

data_dict = {}

BMIBP_datasets = {
    "female" : {
        "BMI" : BMI_female_dataframe,
        "BP" : BP_female_dataframe,
    },
    "male" : {
        "BMI" : BMI_male_dataframe,
        "BP" : BP_male_dataframe,
    }
}

# split by 6 continents and genders
for ctnt in range(1, 7):
    data_dict[ctnt] = {}
    ctnt_df = continent_dataframe[continent_dataframe["Continent"] == ctnt]
    ctnt_cntry_set = set(ctnt_df[ctnt_df.columns[0]])
    
    # urban pct
    data_dict[ctnt]["urban"] = urban_pct_dataframe[
        urban_pct_dataframe[urban_pct_dataframe.columns[0]].isin(ctnt_cntry_set)]
    
    for gender in BMIBP_datasets:
        data_dict[ctnt][gender] = {}
        for data_type in BMIBP_datasets[gender]:
            df = BMIBP_datasets[gender][data_type]
            data_dict[ctnt][gender][data_type] = df[df[df.columns[0]].isin(ctnt_cntry_set)]


In [4]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource, CustomJS
from numpy import nan
from bokeh.models.widgets import Slider
from bokeh.layouts import layout
from ipywidgets import interact
from bokeh.models import HoverTool

output_notebook()

big_data_book = {}

color_book = {
    1 : "Chocolate",
    2 : "Blue",
    3 : "crimson",
    4 : "Gold",
    5 : "LightGreen",
    6 : "DarkOrchid"
}

ctnt_book = {
    1 : "Africa",
    2 : "North America",
    3 : "South America",
    4 : "Asia",
    5 : "Europe",
    6 : "Oceania"
}

start_year = 1980

def set_param(db, ctnt, year):
    db["x"] = db["{}urban{}".format(ctnt, year)]
    db["yf"] = db["{}femaleBP{}".format(ctnt, year)]
    db["ym"] = db["{}maleBP{}".format(ctnt, year)]
    db["sizef"] = db["{}femaleBMI{}".format(ctnt, year)]
    db["sizem"] = db["{}maleBMI{}".format(ctnt, year)]

for ctnt in range(1, 7):
    data_book = {}
    
    # country list
    ctnt_df = continent_dataframe[continent_dataframe["Continent"] == ctnt]
    ctnt_cntry_list = list(ctnt_df[ctnt_df.columns[0]])
    data_book["country"] = ctnt_cntry_list
    
    for year in range(1980, 2009):
        urban = list(data_dict[ctnt]["urban"][year])
        urban_key = "{}{}{}".format(ctnt, "urban", year)
        # nan
        for i in range(len(urban)):
            tmp = urban[i]
            if not (tmp >=0 and tmp <= 100):
                urban[i] = nan
        data_book[urban_key] = urban

        for gender in data_dict[ctnt]:
            if gender != "female" and gender != "male":
                continue
            for data_type in data_dict[ctnt][gender]:
                data_lst = list(data_dict[ctnt][gender][data_type][year])
                if data_type == "BMI":
                    data_lst = [(data - 15) * 1.2 for data in data_lst]
                data_key = "{}{}{}{}".format(ctnt, gender, data_type, year)
                data_book[data_key] = data_lst
                
    set_param(data_book, ctnt, start_year)
    big_data_book[ctnt] = data_book

# plot
source = {}
dots = {}
hover = HoverTool(tooltips=[
    ("Country", "@country"),
])
p = figure(plot_width=900, y_range=[105, 145], tools=[hover])
p.xaxis.axis_label = "Urban population (% of total)"
p.yaxis.axis_label = "Blood pressure (mm Hg)"
for ctnt in range(1, 7):
    source[ctnt] = ColumnDataSource(data=big_data_book[ctnt])
    dots[ctnt] = {}
    
    lf = "Female in {}".format(ctnt_book[ctnt])
    lm = "Male in {}".format(ctnt_book[ctnt])
    
    dots[ctnt]["f"] = p.circle(x="x", y="yf", size="sizef", color=color_book[ctnt], source=source[ctnt], alpha=0.7, legend=lf)
    dots[ctnt]["m"] = p.square(x="x", y="ym", size="sizem", color=color_book[ctnt], source=source[ctnt], alpha=0.7, legend=lm)

# interactive legend
p.legend.location = "bottom_right"
p.legend.click_policy="hide"
p.legend.background_fill_alpha = 0.65
p.legend.label_text_font_size = "7pt"
p.title.text = "Blood pressure / Urbanization chart (Dot size: BMI)"

show(p, notebook_handle=True)
 
# interact
def update(Year = 1980):
    for i in range(1, 7):
        db = big_data_book[i]
        set_param(db, i, Year)
        
        left = dots[i]["f"].data_source.data
        left["x"] = db["x"]
        left["yf"] = db["yf"]
        left["sizef"] = db["sizef"]
        
        left = dots[i]["m"].data_source.data
        left["x"] = db["x"]
        left["ym"] = db["ym"]
        left["sizem"] = db["sizem"]

    push_notebook()



Loading BokehJS ...

In [5]:
widget = interact(update, Year=(1980, 2008, 1))